## Importing a CSV file into a AWS RDS Postgres Database


*   import the csv file into a pandas df
*   clean the table name and remove all extra symbols, spaces, capital letters
*   clean the column header and remove all extra symbols, spaces, capital letters
*   write the create table SQL statement
*   import the data into DB






In [ ]:
!pip install psycopg2-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 4.3 MB/s 


In [ ]:
#import libraries
import os
import numpy as np
import pandas as pd
import psycopg2

In [ ]:
!ls

'Customer Contracts$.csv'   sample_data


In [ ]:
df = pd.read_csv('Customer Contracts$.csv')
df.head()

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


In [ ]:
# clean table name
# all lower case letters
# remove all white spaces and $
# replace -, /, \\, $ with _

file = 'Customer Contracts$'
clean_tbl_name = file.lower().replace(" ","_").replace("?","")\
                    .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","_")\
                    .replace("(","").replace(r")","").replace("$","")

clean_tbl_name

'customer_contracts'

In [ ]:
# clean header name
# all lower case letters
# remove all white spaces and $
# replace -, /, \\, $ with _

# Using List Comprehension
df.columns = [i.lower().replace(" ","_").replace("?","")\
                    .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","_")\
                    .replace("(","").replace(r")","").replace("$","") for i in df.columns]

df.columns

Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

In [ ]:
# SQL datatypes
# create table customer_contracts
# (
#     customer_name   varchar,
#     start_date      varchar,
#     end_date        varchar,
#     contract_amount_m float,
#     invoice_sent   varchar,
#     paid            varchar
#  );

## SQL datatypes

create table customer_contracts
( customer_name   varchar, start_date  varchar,end_date    varchar,contract_amount_m float,invoice_sent   varchar,paid  varchar)

In [ ]:
df.dtypes

customer_name         object
start_date            object
end_date              object
contract_amount_m    float64
invoice_sent          object
paid                  object
dtype: object

In [ ]:
# Create Pandas Datatype to replace SQL datatype
# use dic{}
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
}

In [ ]:
col_str =  ',  '.join("{}  {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'customer_name  varchar,  start_date  varchar,  end_date  varchar,  contract_amount_m  float,  invoice_sent  varchar,  paid  varchar'

In [ ]:
conn_string = "host = rds-postgres.xxxxxxxx.us-east-1.rds.amazonaws.com \
                      db_name = 'rds-postgres' \
                      user = 'postgres' \
                      password = 'xxxxxxxx' "
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print("opened the database successfully")